In [1]:
import numpy as np
import time
import os
%matplotlib qt
import matplotlib.pyplot as pl
import pickle

In [2]:
from sep_decoder import transfer,ht,lp,load_measurement_data,load_transfer_matrices,initial_state

In [3]:
def sep_dynamics(data,Q):
    (depth,L) = data.shape  
    Q2 = Q+1
    if Q>L//2:
        Q2 = Q-1
    hash_table,_ = ht.get_hash_table(L,Q)
    hash_table2,_= ht.get_hash_table(L,Q2)
    pairings = lp.local_pairings(L,Q,hash_table)
    pairings2 = lp.local_pairings(L,Q2,hash_table2)
    dim = len(hash_table)
    dim2 = len(hash_table2)
    
    total = 0

    transfer_matrices = load_transfer_matrices(pairings,dim,L,Q)
    transfer_matrices2 = load_transfer_matrices(pairings2,dim2,L,Q2)
    
    initial_state_Q = np.eye(dim)/dim**0.5
    initial_state_Q2 = np.eye(dim2)/dim2**0.5

    N=1
    traj = data
    state_Q_is_zero = False
    state_Q2_is_zero = False
    state_Q = initial_state_Q.copy()
    state_Q2 = initial_state_Q2.copy()

    log_Z = []
    log_Z2 = []
    total += N
    for t in range(depth)[:]:
        
        if t%2 == 0: #even layer
            for x in range(0,L-1,2):
                outcomes = (traj[t,x],traj[t,x+1])
            
                state_Q, state_Q_is_zero = transfer(x,dim,pairings,transfer_matrices, outcomes,state_Q,log_Z,state_Q_is_zero,L,Q,debug=False)

                state_Q2, state_Q2_is_zero = transfer(x,dim2,pairings2,transfer_matrices2 ,outcomes,state_Q2,log_Z2,state_Q2_is_zero,L,Q2)
                
                if state_Q_is_zero and state_Q2_is_zero:
                    print('hurray2',np.sum(state_Q2),np.sum(state_Q))
        else:
            for x in range(1,L-1,2):
                outcomes = (traj[t,x],traj[t,x+1])
                debug=False

                state_Q, state_Q_is_zero = transfer(x,dim,pairings,transfer_matrices, outcomes,state_Q,log_Z,state_Q_is_zero,L,Q,debug=False)

                state_Q2, state_Q2_is_zero = transfer(x,dim2,pairings2,transfer_matrices2 ,outcomes,state_Q2,log_Z2,state_Q2_is_zero,L,Q2)

                if state_Q_is_zero and state_Q2_is_zero:
                    print('hurray2',np.sum(state_Q2),np.sum(state_Q))
                

        # print(t,time.time()-start,total)
        if state_Q_is_zero:              
            break
    
        if state_Q2_is_zero:
            break
        
    if state_Q_is_zero:
        p_Q = 0
    elif state_Q2_is_zero:
        p_Q = 1
    else:
        ratio_p = np.exp(np.sum(log_Z) - np.sum(log_Z2))
        p_Q = 1/(1+1/ratio_p)
        p_Q2 = 1/(1+ratio_p)
    
    return p_Q

In [4]:
def get_p_L(filename):
    temp = filename.split(',')
    p = float(temp[-1][:-4])
    L = int(temp[1])
    return p,L

filedir = 'U_1_data_Fergus/'
p_set = set()
for file in os.listdir(filedir)[:]:
    if file[-4:] != '.npy':
        continue
    temp = file.split(',')
    p,L = get_p_L(file)
    p_set.add(p)
p_list = list(sorted(p_set))[::]
print(p_list,len(p_list),sorted(p_set))

[0.025, 0.0375, 0.05, 0.0625, 0.075, 0.0875, 0.1, 0.1125, 0.125, 0.1375, 0.15, 0.1625, 0.175, 0.1875, 0.2, 0.2125, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.4, 0.45, 0.5] 25 [0.025, 0.0375, 0.05, 0.0625, 0.075, 0.0875, 0.1, 0.1125, 0.125, 0.1375, 0.15, 0.1625, 0.175, 0.1875, 0.2, 0.2125, 0.225, 0.25, 0.275, 0.3, 0.325, 0.35, 0.4, 0.45, 0.5]


In [5]:
# L_list = [6,8,10,12]
# for L in L_list:
#     for p in p_list:
#         for file in os.listdir(filedir)[:]:
#             if file[-4:] != '.npy':
#                 continue
#             p_data,L_data = get_p_L(file)
#             if p_data != p or L_data != L:
#                 continue
#             shutil.copy(filedir+file,filedir+'useful/'+file)

In [22]:
def get_sep_data(N_samples,L,p):
    p_suc = {}
    for file in os.listdir(filedir)[:]:
        if file[-4:] != '.npy':
            continue
        p_data,L_data = get_p_L(file)
        if p_data != p or L_data != L:
            continue
        data_raw = np.load(filedir+file)[:N_samples]
        for i in [0,1]:
            Q = int(np.sum(0.5*(data_raw[0,i,:,-1]+1)))
            p_suc[Q] = []
            for j in range(N_samples):
                p_suc[Q].append(sep_dynamics_2(data_raw[j,i,:,:-1].T,Q))
    return p_suc



In [60]:
%time
L_list = [6,8,10,12]
p_suc_dic = {}
for L in L_list:
    p_suc_dic[L]={}
    for p in p_list[:]:
        start = time.time()
        p_suc_dic[L][p]=get_sep_data(3000,L,p)
        print(L,p,time.time()-start)


CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 2.86 µs
6 0.025 5.716858148574829
6 0.0375 5.718504190444946
6 0.05 5.6886022090911865
6 0.0625 5.693036079406738
6 0.075 5.691883087158203
6 0.0875 5.658500909805298
6 0.1 5.652005910873413
6 0.1125 5.724509239196777
6 0.125 5.615649938583374
6 0.1375 5.577239036560059
6 0.15 5.52669882774353
6 0.1625 5.532447814941406
6 0.175 5.567655801773071
6 0.1875 5.462685823440552
6 0.2 5.388970136642456
6 0.2125 5.319613933563232
6 0.225 5.248443126678467
6 0.25 5.043322801589966
6 0.275 5.120929002761841
6 0.3 4.856543779373169
6 0.325 4.410391092300415
6 0.35 4.226274013519287
6 0.4 3.6510400772094727
6 0.45 3.1519479751586914
6 0.5 2.746528148651123
8 0.025 11.975646257400513
8 0.0375 11.785084247589111
8 0.05 11.785102128982544
8 0.0625 11.909101247787476
8 0.075 13.65444016456604
8 0.0875 12.317476272583008
8 0.1 12.130368947982788
8 0.1125 12.127177953720093
8 0.125 12.095174074172974
8 0.1375 11.80032992362976
8 0.15 11.646

In [62]:
pl.figure(1)
plot_success_ent(p_suc_dic,L_list,p_list[:])

L= 6 p= 0.025 data_size: 2999 2999
L= 6 p= 0.0375 data_size: 2999 2999
L= 6 p= 0.05 data_size: 2999 2999
L= 6 p= 0.0625 data_size: 2999 2999
L= 6 p= 0.075 data_size: 2999 2999
L= 6 p= 0.0875 data_size: 2999 2999
L= 6 p= 0.1 data_size: 2999 2999
L= 6 p= 0.1125 data_size: 2999 2999
L= 6 p= 0.125 data_size: 2999 2999
L= 6 p= 0.1375 data_size: 2999 2999
L= 6 p= 0.15 data_size: 2999 2999
L= 6 p= 0.1625 data_size: 2999 2999
L= 6 p= 0.175 data_size: 2999 2999
L= 6 p= 0.1875 data_size: 2999 2999
L= 6 p= 0.2 data_size: 2999 2999
L= 6 p= 0.2125 data_size: 2999 2999
L= 6 p= 0.225 data_size: 2999 2999
L= 6 p= 0.25 data_size: 2999 2999
L= 6 p= 0.275 data_size: 2999 2999
L= 6 p= 0.3 data_size: 2999 2999
L= 6 p= 0.325 data_size: 2999 2999
L= 6 p= 0.35 data_size: 2999 2999
L= 6 p= 0.4 data_size: 2999 2999
L= 6 p= 0.45 data_size: 2999 2999
L= 6 p= 0.5 data_size: 2999 2999
L= 8 p= 0.025 data_size: 2999 2999
L= 8 p= 0.0375 data_size: 2999 2999
L= 8 p= 0.05 data_size: 2999 2999
L= 8 p= 0.0625 data_size: 2

In [8]:
def plot_success_ent(data,L_list,p_list,N_samples=-1):
    ent = {}
    err = {}
    for L in L_list[:]:
        ent[L] = []
        err[L] = []
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2+1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = [(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ]
            # ent_list = [-np.log(x**2+(1-x)**2) if 0<x<1 else 0 for x in tempQ2+tempQ]
            ent[L].append(np.average(ent_list))
            err[L].append(np.std(ent_list)/(len(ent_list)-1)**0.5)

        pl.errorbar(p_list,ent[L],yerr=err[L],ls='-',marker='o',label='L='+str(L))

    pl.xlabel(r'$p$',fontsize=16)
    pl.ylabel(r'$S_{suc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

In [73]:
def plot_success_binder(data,L_list,p_list,N_samples=-1):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        for p in p_list:
            tempQ = list(data[L][p][L//2])[1000:1000+N_samples]
            tempQ2 = list(data[L][p][L//2+1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = np.array([(-x*np.log(x) - (1-x)*np.log(1-x)) if 0<x<1 else 0 for x in tempQ2+tempQ])
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            binder[L].append(1-mu_4/(3*mu_2**2))
            err[L].append((e_4/mu_2 + mu_4*e_2/(mu_2**2)))

        pl.errorbar(p_list,binder[L],yerr=err[L],ls='-',marker='o',label='L='+str(L))

    pl.xlabel(r'$p$',fontsize=16)
    pl.ylabel(r'Binder${}_{suc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

def plot_acc_binder(data,L_list,p_list,N_samples=-1):
    binder = {}
    err = {}
    for L in L_list[:]:
        binder[L] = []
        err[L] = []
        for p in p_list:
            tempQ = list(data[L][p][L//2])[:N_samples]
            tempQ2 = list(data[L][p][L//2+1])[:N_samples]
            print("L=",L,"p=",p,"data_size:",len(tempQ),len(tempQ2))
            # ent[L].append(np.sum(np.array(tempQ+tempQ2)>0.5)/len(tempQ+tempQ2))
            # err[L].append(0)
            ent_list = np.array([x for x in tempQ2+tempQ])
            mu_1 = np.average(ent_list)
            mu_4 = np.average((ent_list-mu_1)**4)
            e_4 = np.std(ent_list**4)/len(tempQ+tempQ2)
            mu_2 = np.average((ent_list-mu_1)**2)
            e_2 = 2*mu_2*np.std(ent_list**2)/len(tempQ+tempQ2)
            binder[L].append(1-mu_4/(3*mu_2**2))
            err[L].append((e_4/mu_2 + mu_4*e_2/(mu_2**2)))

        pl.errorbar(p_list,binder[L],yerr=err[L],ls='-',marker='o',label='L='+str(L))

    pl.xlabel(r'$p$',fontsize=16)
    pl.ylabel(r'Binder${}_{acc}$',fontsize=16)
    pl.legend(fontsize=16)
    pl.tight_layout()

In [74]:
L_list = [6,8,10,12]
pl.figure(2)
with open('U_1_data_Fergus/sep_data_2','rb') as f:
    data = pickle.load(f)
plot_success_ent(data,L_list[:],p_list[:],N_samples=400)
pl.grid(which='both')

L= 6 p= 0.025 data_size: 400 400
L= 6 p= 0.0375 data_size: 400 400
L= 6 p= 0.05 data_size: 400 400
L= 6 p= 0.0625 data_size: 400 400
L= 6 p= 0.075 data_size: 400 400
L= 6 p= 0.0875 data_size: 400 400
L= 6 p= 0.1 data_size: 400 400
L= 6 p= 0.1125 data_size: 400 400
L= 6 p= 0.125 data_size: 400 400
L= 6 p= 0.1375 data_size: 400 400
L= 6 p= 0.15 data_size: 400 400
L= 6 p= 0.1625 data_size: 400 400
L= 6 p= 0.175 data_size: 400 400
L= 6 p= 0.1875 data_size: 400 400
L= 6 p= 0.2 data_size: 400 400
L= 6 p= 0.2125 data_size: 400 400
L= 6 p= 0.225 data_size: 400 400
L= 6 p= 0.25 data_size: 400 400
L= 6 p= 0.275 data_size: 400 400
L= 6 p= 0.3 data_size: 400 400
L= 6 p= 0.325 data_size: 400 400
L= 6 p= 0.35 data_size: 400 400
L= 6 p= 0.4 data_size: 400 400
L= 6 p= 0.45 data_size: 400 400
L= 6 p= 0.5 data_size: 400 400
L= 8 p= 0.025 data_size: 400 400
L= 8 p= 0.0375 data_size: 400 400
L= 8 p= 0.05 data_size: 400 400
L= 8 p= 0.0625 data_size: 400 400
L= 8 p= 0.075 data_size: 400 400
L= 8 p= 0.0875 d

qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 2 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 1 - updating layer to match.
qt.qpa.backingstore: Back buffer dpr of 1 doesn't match <_NSViewBackingLayer: 0x2dd10be50> contents scale of 2 - updating layer to match.
qt.qpa.backingstore: Back buffer d

In [11]:
850*25/3600

5.902777777777778

In [12]:
from sep_decoder_2 import sep_dynamics_2

In [20]:
def get_sep_data_comparision(N_samples,L,p):
    p_suc = {}
    p_suc_2 = {}
    for file in os.listdir(filedir)[:]:
        if file[-4:] != '.npy':
            continue
        p_data,L_data = get_p_L(file)
        if p_data != p or L_data != L:
            continue
        data_raw = np.load(filedir+file)[:N_samples]
        for i in [0,1]:
            Q = int(np.sum(0.5*(data_raw[0,i,:,-1]+1)))
            p_suc[Q] = []
            p_suc_2[Q] = []
            for j in range(N_samples):
                # p_suc[Q].append(sep_dynamics(data_raw[j,i,:,:-1].T,Q))
                p_suc_2[Q].append(sep_dynamics_2(data_raw[j,i,:,:-1].T,Q))
    return p_suc,p_suc_2


In [21]:
%timeit p_suc1,p_suc2 = get_sep_data_comparision(10,12,0.2)

160 ms ± 825 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
np.round(p_suc1[7],3)-np.round(p_suc2[7],3)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])